In [30]:
import sys
import os

# 将项目根目录添加到 sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))  # 跳出 src/langchainAgenticAi/retrieval
sys.path.append(project_root)

In [31]:
import os
import glob
import logging
# from langchainAgenticAi.utils import build_index, split_documents, recall_documents
from src.langchainAgenticAi.utils.base_retrieval import build_index, split_documents, recall_documents 

from langchain_community.vectorstores import Chroma
import fitz  # 用于PDF文件处理

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 从环境变量中获取本地知识库的路径
local_knowledge_base_path = os.getenv('LOCAL_KNOWLEDGE_BASE_PATH', 'Local_knowledge_base')

In [32]:
def load_pdf_from_folder(folder_path):
    """
    加载文件夹中的所有 PDF 文件，并提取文本内容
    """
    all_documents = []
    for filename in glob.glob(os.path.join(folder_path, "*.pdf")):
        document_text = ""
        with fitz.open(filename) as doc:
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                document_text += page.get_text("text")
        all_documents.append(document_text)
    return all_documents

In [33]:
def local_retrieve(query):
    """
    使用本地知识库进行检索
    """
    # 加载本地知识库中的 PDF 文档
    documents = load_pdf_from_folder(local_knowledge_base_path)
    
    # 对文档进行分片
    split_docs = split_documents(documents)
    
    # 构建本地向量索引
    index = build_index(split_docs)
    
    # 执行检索
    results = recall_documents(query, index)
    
    return results